In [9]:
!tar -xvzf ohsumed-first-20000-docs.tar.gz

ohsumed-first-20000-docs/
ohsumed-first-20000-docs/training/
ohsumed-first-20000-docs/training/C01/
ohsumed-first-20000-docs/training/C01/0000021
ohsumed-first-20000-docs/training/C01/0000033
ohsumed-first-20000-docs/training/C01/0000052
ohsumed-first-20000-docs/training/C01/0000306
ohsumed-first-20000-docs/training/C01/0000334
ohsumed-first-20000-docs/training/C01/0000335
ohsumed-first-20000-docs/training/C01/0000352
ohsumed-first-20000-docs/training/C01/0000356
ohsumed-first-20000-docs/training/C01/0000358
ohsumed-first-20000-docs/training/C01/0000391
ohsumed-first-20000-docs/training/C01/0000418
ohsumed-first-20000-docs/training/C01/0000434
ohsumed-first-20000-docs/training/C01/0000436
ohsumed-first-20000-docs/training/C01/0000569
ohsumed-first-20000-docs/training/C01/0000571
ohsumed-first-20000-docs/training/C01/0000636
ohsumed-first-20000-docs/training/C01/0000642
ohsumed-first-20000-docs/training/C01/0000684
ohsumed-first-20000-docs/training/C01/0000685
ohsumed-first-20000-docs/t

In [10]:
import os
import io
import math
import nltk
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding


from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer


Using TensorFlow backend.


In [11]:

pathTrainingDir = 'ohsumed-first-20000-docs/training/'
pathTestDir = 'ohsumed-first-20000-docs/test/'

#stop_words = set(stopwords.words('english')).union(set(list(punctuation)))

listCategories = os.listdir(pathTrainingDir)
# Dictionnaire qui associe chaque categorie aux différents articles qui font référence à cette dernière
dictCatgoriesArticles_training = {}
dictCategoriesArticles_test = {}

for categorie in listCategories :
  Newpath = pathTrainingDir+categorie+'/'
  NewpathTest = pathTestDir+categorie+'/'
  # Liste des fichiers contenus dans une categorie
  articlesTraining = os.listdir(Newpath)
  articlestest = os.listdir(NewpathTest)
  
  #categorie = categorie.replace('C','')
  dictCatgoriesArticles_training[categorie] = articlesTraining
  dictCategoriesArticles_test[categorie] = articlestest;


   
print(dictCatgoriesArticles_training.items())



dict_items([('C13', ['0007743', '0005547', '0006412', '0004657', '0008308', '0002531', '0005527', '0007065', '0005921', '0009600', '0002676', '0004303', '0004296', '0007937', '0007925', '0003906', '0007113', '0002527', '0004960', '0006415', '0006445', '0004216', '0008266', '0003112', '0008317', '0002551', '0002388', '0006510', '0006452', '0003912', '0008315', '0007926', '0008526', '0005513', '0003922', '0003100', '0009852', '0004969', '0002546', '0007935', '0007061', '0005515', '0007958', '0002457', '0006418', '0003377', '0006454', '0004266', '0000996', '0007934', '0003902', '0006016', '0008290', '0008873', '0009935', '0003926', '0003369', '0006925', '0005256', '0003249', '0004719', '0005529', '0004804', '0007440', '0006444', '0005535', '0002529', '0005534', '0003920', '0008902', '0007105', '0006441', '0003083', '0005524', '0009139', '0003245', '0009857', '0008373', '0006820', '0006435', '0006440', '0004958', '0006300', '0008376', '0003770', '0003250', '0002525', '0002980', '0004891', 

#   Liaison entre les articles et leur contenu

In [12]:
# Pour le training part
dictTraining_ArticlesText = {}
for key in dictCatgoriesArticles_training.keys() : 
  allArticles = dictCatgoriesArticles_training[key] # on recupère tous les articles de la categorie
  for article in allArticles :
    FilePath = pathTrainingDir + key+'/'+article
    fileContent = ""
    with io.open(FilePath, "r", newline=None) as fd:
      for line in fd:
        line = line.replace("\n", "")
        fileContent = fileContent + line
    
    dictTraining_ArticlesText[article] = fileContent;    


  
# Pour le test part
dictTest_ArticlesText = {}

for key in dictCategoriesArticles_test.keys() : 
  allArticles = dictCategoriesArticles_test[key] # on recupère tous les articles de la categorie
  for article in allArticles :
    FilePath = pathTestDir + key +'/'+ article
    #file = open(FilePath, 'r') 
    fileContent = ""
    with io.open(FilePath, "r", newline=None) as fd:
      for line in fd:
        line = line.replace("\n", "")
        fileContent = fileContent + line
        
    dictTest_ArticlesText[article] = fileContent;    
  #  print(fileContent)
 #   break
 # break
print(dictTest_ArticlesText)




IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Liaison de chaque article aux différentes catégories auxquelles il appartient
Sachant qu'un même article peut se retrouver dans différentes catégories donc on fait un dictionnaire dont la clé est le nom de l'article et la valeur une liste des catégories auxquelles ce dernier appartient. On élimine en même temps le caractère C dans le nom de la catégorie afin de garder que des entiers. par exemple, la catégorie C1 sera representée par 1..

In [13]:
# Pour les articles du training data
dict_article_categories_training = {}

for cat, articles in dictCatgoriesArticles_training.items():
  for article in articles : 
    if article in dict_article_categories_training.keys():
      dict_article_categories_training[article].append(int(cat.replace("C", "")))
    else :
      dict_article_categories_training[article] = [int(cat.replace("C", ""))]
        
print(dict_article_categories_training)
  
# Pour les articles du test data
dict_article_categories_test = {}

for cat, articles in dictCategoriesArticles_test.items():
  for article in articles : 
    if article in dict_article_categories_test.keys():
      dict_article_categories_test[article].append(int(cat.replace("C", "")))
    else :
      dict_article_categories_test[article] = [int(cat.replace("C", ""))]
        
print(dict_article_categories_test)
  

{'0007743': [13, 14], '0005547': [13, 23], '0006412': [13], '0004657': [13, 23, 1, 12], '0008308': [13, 2, 23, 1, 12, 20], '0002531': [13, 4], '0005527': [13], '0007065': [13, 23], '0005921': [13, 1], '0009600': [13, 19, 20], '0002676': [13], '0004303': [13], '0004296': [13, 12], '0007937': [13], '0007925': [13], '0003906': [13], '0007113': [13, 23, 16, 10, 4], '0002527': [13, 15], '0004960': [13], '0006415': [13, 1], '0006445': [13, 12], '0004216': [13], '0008266': [13, 23], '0003112': [13], '0008317': [13, 1], '0002551': [13, 6, 21], '0002388': [13, 4], '0006510': [13], '0006452': [13, 4], '0003912': [13], '0008315': [13, 23, 1, 12], '0007926': [13], '0008526': [13, 4], '0005513': [13, 4], '0003922': [13], '0003100': [13], '0009852': [13], '0004969': [13, 10], '0002546': [13, 2, 1, 3, 18], '0007935': [13, 12], '0007061': [13], '0005515': [13, 4], '0007958': [13], '0002457': [13, 12, 4], '0006418': [13], '0003377': [13, 12, 4], '0006454': [13, 23], '0004266': [13], '0000996': [13, 23,

# Labelisation des différentes catégories 
On choisit d'encapsuler chaque article dans un vecteur de taille 23 (correspondant au nombre de catégories) et on met à 1 à chaque fois que l'article appartient à une catégorie donnée.

In [14]:
import numpy as np
# initilisation d'un vecteur de taille 23 à zero
vecteur = []
for i in range(23):
  vecteur.append(0)
  
# Partie training

y_train = []


for listCategorie in dict_article_categories_training.keys():
  vect_copy = vecteur.copy()
  labels = dict_article_categories_training[listCategorie] #labels 
  for label in labels :
    vect_copy[label-1] = 1
  y_train.append(vect_copy)
  

# Partie test


y_test = []


for listCategorie in dict_article_categories_test.keys():
  vect_copy = vecteur.copy()
  labels = dict_article_categories_test[listCategorie] #labels 
  for label in labels :
    vect_copy[label-1] = 1
  y_test.append(vect_copy)
  
# Transformation des listes de labels en des numpy array
y_train = np.asarray(y_train)
y_test  = np.asarray(y_test)
    
print(len(vecteur))
print(y_train)
print(y_test)

23
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# Preprocessing data

In [20]:
# Recuperation de tous les contenus des articles pour le train et pour le test


x_train_text = list()
x_train_text_temp = list()

x_test_text = list()
x_test_text_temp = list()

for key,text in dictTest_ArticlesText.items():
  x_test_text_temp.append(text)
  
for key,text in dictTraining_ArticlesText.items():
  x_train_text_temp.append(text)
  
t = Tokenizer()

# fit the tokenizer on the documents
#t.fit_on_texts(x_test_text_temp + x_train_text_temp)
    
x_test_text = t.texts_to_sequences(x_test_text_temp)
x_train_text = t.texts_to_sequences(x_train_text_temp)
  
x_train_text = sequence.pad_sequences(x_train_text, maxlen=420)
x_test_text = sequence.pad_sequences(x_test_text, maxlen=420)
  
print('x_train_text shape:', x_train_text.shape)
print('x_test_text shape:', x_test_text.shape)

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train_text shape: (6286, 420)
x_test_text shape: (7643, 420)
y_train shape: (6286, 23)
y_test shape: (7643, 23)


# Building the model


In [25]:
from keras.layers import LSTM

max_features = 300000
# cut texts after this number of words (among top max_features most common words)
batch_size = 24

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(48, dropout=0.5, recurrent_dropout=0.3))
model.add(Dense(23, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train_text, y_train,
          batch_size=batch_size,
          epochs=5,
          validation_data=(x_test_text, y_test))
score, acc = model.evaluate(x_test_text, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Build model...
Train...
Train on 6286 samples, validate on 7643 samples
Epoch 1/5
6286/6286 [==============================] - 131s 21ms/step - loss: 0.2442 - acc: 0.9278 - val_loss: 0.2438 - val_acc: 0.9276
Epoch 2/5
6286/6286 [==============================] - 135s 21ms/step - loss: 0.2415 - acc: 0.9278 - val_loss: 0.2435 - val_acc: 0.9276
Epoch 3/5
6286/6286 [==============================] - 134s 21ms/step - loss: 0.2415 - acc: 0.9278 - val_loss: 0.2433 - val_acc: 0.9276
Epoch 4/5
6286/6286 [==============================] - 134s 21ms/step - loss: 0.2415 - acc: 0.9278 - val_loss: 0.2436 - val_acc: 0.9276
Epoch 5/5
7643/7643 [==============================] - 15s 2ms/step
Test score: 0.24358691790996695
Test accuracy: 0.9275665795726644


# Conclusion

Après la construction du model on constate qu'un obtient un accurracy de 92% .